In [ ]:
from glob import glob
import os
import pandas as pd

In [ ]:
def parse_name(df):
    df[['n', 't', 'd', 'c', 'idx', 'r']] = df['name'].str.extract(r'cap100_n(\d+)_t(\d+)_(Lon|Sh)(Lr|Sm)_(\d+)_(\d+)\.txt')
    df['n'] = df['n'].astype(int)
    df['t'] = df['t'].astype(int)
    df['d'] = df['d'].map(lambda x: {'Lon': 'd_L', 'Sh': 'd_S'}[x])
    df['c'] = df['c'].map(lambda x: {'Lr': 'c_H', 'Sm': 'c_L'}[x])
    df['idx'] = df['idx'].astype(int)
    df['r'] = df['r'].astype(int)
    df['rat'] = (df['r'] / df['n']).map(lambda x: f'{x:.2f}')

In [ ]:
models = {
    'model1n_bpp_wy_heu_s': 'M1**',
    'model1ts_bpp_wy_heu_s': 'M1**+(R0)',
}

In [ ]:
logs = []
for file in glob('../logs/mip_r/*.log'):
    log = pd.read_csv(file, sep='\t')
    log['model'] = os.path.basename(file)[:-4].split('_', 1)[1]
    logs.append(log)

df = pd.concat(logs)
parse_name(df)
df['dt_solve'] = df['dt_solve'].clip(0.0, 1800.0)

# Table 4

In [ ]:
pt = df.pivot_table(
    index=['n'],
    columns=['rat', 'model'],
    values=['mrel.ObjVal'],
).sort_index()
pt.loc['Total'] = pt.mean(axis=0)
pt.rename(columns=models).style.format('{:.1f}')

# Tables 5 and 6

In [ ]:
pt = df.pivot_table(
    index=['n'],
    columns=['rat', 'model'],
    aggfunc={'dt_solve': 'mean', 'optimal': 'sum'}
).sort_index().reorder_levels([1, 2, 0], axis='columns').sort_index(axis='columns')

total = pt.mean(axis=0)
total[1::2] = pt.sum(axis=0)[1::2]
pt.loc['Total'] = total
pt = pt.rename(columns=models)

pt.style.format('{:.1f}', subset=pt.columns[::2]).format('({:.0f})', subset=pt.columns[1::2])